In [1]:
import jittor as jt
from jittor import init
import argparse
import os
import numpy as np
import math
from jittor import nn

if jt.has_cuda:
    jt.flags.use_cuda = 1


C:\Users\zxy08\.cache\jittor\jtcuda\cuda11.2_cudnn8_win\bin\nvcc.exe


In [2]:
import sys
sys.argv = ['run.py']
    

In [3]:
parser = argparse.ArgumentParser()
parser.add_argument('--n_epochs', type=int, default=100, help='number of epochs of training')
parser.add_argument('--batch_size', type=int, default=64, help='size of the batches')
parser.add_argument('--lr', type=float, default=0.0002, help='adam: learning rate')
parser.add_argument('--b1', type=float, default=0.5, help='adam: decay of first order momentum of gradient')
parser.add_argument('--b2', type=float, default=0.999, help='adam: decay of first order momentum of gradient')
parser.add_argument('--n_cpu', type=int, default=8, help='number of cpu threads to use during batch generation')
parser.add_argument('--latent_dim', type=int, default=100, help='dimensionality of the latent space')
parser.add_argument('--n_classes', type=int, default=10, help='number of classes for dataset')
parser.add_argument('--img_size', type=int, default=32, help='size of each image dimension')
parser.add_argument('--channels', type=int, default=1, help='number of image channels')
parser.add_argument('--sample_interval', type=int, default=1000, help='interval between image sampling')
opt = parser.parse_args()
print(opt)

Namespace(n_epochs=100, batch_size=64, lr=0.0002, b1=0.5, b2=0.999, n_cpu=8, latent_dim=100, n_classes=10, img_size=32, channels=1, sample_interval=1000)


In [4]:
img_shape = (opt.channels, opt.img_size, opt.img_size)

In [25]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.label_emb = nn.Embedding(opt.n_classes, opt.n_classes)
        # nn.Linear(in_dim, out_dim)表示全连接层
        # in_dim：输入向量维度
        # out_dim：输出向量维度
        def block(in_feat, out_feat, normalize=True):  #用于定义一个层
            layers = [nn.Linear(in_feat, out_feat)]
            if normalize:
                layers.append(nn.BatchNorm1d(out_feat, 0.8))# 0.8是momentum参数，控制均值和方差的移动平均值的权重
            layers.append(nn.LeakyReLU(0.2)) #激活函数是ReLu的变种，当输入小于0时，Leaky ReLU会乘以0.2，而不是直接输出0
            return layers
        self.model = nn.Sequential(*block((opt.latent_dim + opt.n_classes), 128, normalize=False), 
                                   *block(128, 256), 
                                   *block(256, 512), 
                                   *block(512, 1024), 
                                   nn.Linear(1024, int(np.prod(img_shape))), 
                                   nn.Tanh())

    def execute(self, noise, labels):
        gen_input = jt.contrib.concat((self.label_emb(labels), noise), dim=1)
        img = self.model(gen_input)
        # 将img从1024维向量变为32*32矩阵
        img = img.view((img.shape[0], *img_shape))
        return img

In [5]:
test = nn.Embedding(opt.n_classes, opt.n_classes*opt.n_classes)

In [6]:
x = jt.int32([1])

In [8]:
y = test(x)

In [9]:
y.shape

[1,100,]

In [12]:
z = jt.array(np.random.normal(0, 1, (64, opt.latent_dim))).float32() #随机的噪声
gen_labels = jt.array(np.random.randint(0, opt.n_classes, 64)).float32()

In [44]:
gen_labels.shape

[64,]

In [15]:
label = test(gen_labels)

In [16]:
label.shape

[64,100,]

In [41]:
t1 = jt.contrib.concat((label,z), dim=1)

In [42]:
t1.shape

[64,200,]

In [22]:
em = nn.Embedding(opt.n_classes, opt.img_size * opt.img_size)

In [23]:
class Discriminator(nn.Module):

    def __init__(self):
        super(Discriminator, self).__init__()
        self.label_embedding = nn.Embedding(opt.n_classes, opt.n_classes)
        self.model = nn.Sequential(nn.Linear((opt.n_classes + int(np.prod(img_shape))), 512), 
                                   nn.LeakyReLU(0.2), 
                                   nn.Linear(512, 512), 
                                   nn.Dropout(0.4), 
                                   nn.LeakyReLU(0.2), 
                                   nn.Linear(512, 512), 
                                   nn.Dropout(0.4), 
                                   nn.LeakyReLU(0.2), 
                                   # TODO: 添加最后一个线性层，最终输出为一个实数
                                   nn.Linear(512, 1)
                                   )

    def execute(self, img, labels):
        d_in = jt.contrib.concat((img.view((img.shape[0], (- 1))), self.label_embedding(labels)), dim=1)
        # TODO: 将d_in输入到模型中并返回计算结果
        validity = self.model(d_in)
        return validity


In [26]:
# 损失函数：平方误差
# 调用方法：adversarial_loss(网络输出A, 分类标签B)
# 计算结果：(A-B)^2
adversarial_loss = nn.MSELoss()

generator = Generator()
discriminator = Discriminator()

# 导入MNIST数据集
from jittor.dataset.mnist import MNIST
import jittor.transform as transform
transform = transform.Compose([
    transform.Resize(opt.img_size),
    transform.Gray(),
    transform.ImageNormalize(mean=[0.5], std=[0.5]),
])
dataloader = MNIST(train=True, transform=transform).set_attrs(batch_size=opt.batch_size, shuffle=True)

optimizer_G = nn.Adam(generator.parameters(), lr=opt.lr, betas=(opt.b1, opt.b2))
optimizer_D = nn.Adam(discriminator.parameters(), lr=opt.lr, betas=(opt.b1, opt.b2))

In [40]:
for i, (imgs, labels) in enumerate(dataloader):
    real_imgs = jt.array(imgs)
    labels = jt.array(labels)
    print(real_imgs.shape)
    t1 = em(labels)
    t1 = t1.reshape(t1.shape[0], 1, opt.img_size, opt.img_size)
    print(t1.shape)
    t2 = jt.contrib.concat((t1, real_imgs), dim=1)
    print(t2.shape)
    # print(real_imgs[0][0][0])
    # print(t1[0][0][0])
    # print(t2[0][0][0])
    break

[64,1,32,32,]
[64,1,32,32,]
[64,2,32,32,]


In [11]:
L1 = nn.Linear((opt.n_classes + int(np.prod(img_shape))), 1024)
C1 = nn.Conv(1,6,5,1) # 6*28*28
P1 = nn.Pool(2,2) # 6*14*14
C2 = nn.Conv(6,16,5,1) # 16*10*10
P2 = nn.Pool(2,2) # 16*5*5
L2 = nn.Linear(16*5*5, 120)
L3 = nn.Linear(120, 84)
L4 = nn.Linear(84, 10)
embed = nn.Embedding(opt.n_classes, opt.n_classes)

In [8]:
class Flatten(nn.Module):
    def execute(self, x):
        return x.view(x.size(0), -1)
    
class Reshape(nn.Module):
    def execute(self, x):
        return x.view(x.size(0), 1, 32, 32)


In [9]:
model = nn.Sequential(nn.Linear((opt.n_classes + int(np.prod(img_shape))), 1024), #输入维度为图像向量维度+类别数（即label）
                                   nn.LeakyReLU(0.2), #激活函数，当输入小于0时，Leaky ReLU会乘以0.2，而不是直接输出0
                                   Reshape(), #将1024维的向量重新构建成32*32的图像矩阵
                                   nn.Conv(1,6,5,1), # 6*28*28
                                   nn.LeakyReLU(0.2),
                                   nn.Pool(2,2), #  6*14*14
                                   nn.Conv(6,16,5,1), # 16*10*
                                   nn.LeakyReLU(0.2),
                                   nn.Pool(2,2), # 16*5*5
                                   Flatten(), #将图像矩阵展平
                                   nn.Linear(16*5*5, 120),
                                   nn.LeakyReLU(0.2),
                                   nn.Linear(120, 84),
                                   nn.LeakyReLU(0.2),
                                   nn.Linear(84, 1),
                                   nn.Sigmoid() #归一
                                   )

In [12]:
for i, (imgs, labels) in enumerate(dataloader):
    batch_size = imgs.shape[0] #64

    print(imgs.shape, labels.shape)

    print(imgs.view((imgs.shape[0], (- 1))).shape)

    d_in = jt.contrib.concat((imgs.view((imgs.shape[0], (- 1))), embed(labels)), dim=1)
    print(d_in.shape)

    t8 = model(d_in)
    print(t8.shape)
    print(t8)

    # 数据标签，valid=1表示真实的图片，fake=0表示生成的图片
    valid = jt.ones([batch_size, 1]).float32().stop_grad()
    fake = jt.zeros([batch_size, 1]).float32().stop_grad()

    if i==0:
        break


[64,1,32,32,] [64,]
[64,1024,]
[64,1034,]
[64,1,]
jt.Var([[0.4775455 ]
        [0.47970298]
        [0.47715354]
        [0.47776645]
        [0.47834095]
        [0.47867146]
        [0.477492  ]
        [0.47901678]
        [0.478849  ]
        [0.47909257]
        [0.47740248]
        [0.47768167]
        [0.47900912]
        [0.47804525]
        [0.48083615]
        [0.47759145]
        [0.4801024 ]
        [0.47777668]
        [0.48016527]
        [0.4768419 ]
        [0.47751984]
        [0.47815806]
        [0.47764587]
        [0.4784174 ]
        [0.47830474]
        [0.4789069 ]
        [0.47900996]
        [0.4758968 ]
        [0.47781575]
        [0.47745427]
        [0.4766364 ]
        [0.47958088]
        [0.47970286]
        [0.4796728 ]
        [0.47823134]
        [0.47699866]
        [0.47812012]
        [0.47939122]
        [0.47638866]
        [0.47748902]
        [0.4782316 ]
        [0.47879985]
        [0.47901782]
        [0.47712362]
        [0.47959754]
     